In [ ]:
# from pathlib import Path
# import pandas as pd
# import re

# # =========================
# # USER CONFIG
# # =========================
# IN_PATH  = Path("/Users/lvmeizhong/Downloads/edge_diamond_glide_set_dissociated.lmp")  # 改成你的路径
# OUT_PATH = IN_PATH.with_name(IN_PATH.stem + "_swapYZ.lmp")

# PAD = 1e-6          # 给 bounds 加一点点余量，防浮点误差
# FLOAT_FMT = "%.13f" # 输出坐标精度
# # =========================

# # --- 1) 找到 Atoms 段起始行号（用于 skiprows） ---
# with IN_PATH.open("r") as f:
#     lines = f.readlines()

# atoms_hdr_idx = None
# for i, l in enumerate(lines):
#     if l.strip().startswith("Atoms"):
#         atoms_hdr_idx = i
#         break
# assert atoms_hdr_idx is not None, "没找到 'Atoms' 段"

# # Atoms header 下一行通常是空行；原子数据从 atoms_hdr_idx+2 开始
# atoms_data_start = atoms_hdr_idx + 2

# # --- 2) 读 Atoms（你的文件是 Atoms # atomic: id type x y z） ---
# df = pd.read_csv(
#     IN_PATH,
#     delim_whitespace=True,
#     header=None,
#     names=["id", "type", "x", "y", "z"],
#     skiprows=atoms_data_start,
#     comment="#",
#     engine="python",
# )

# # --- 3) swap：把现在的 Y 当 Z，把现在的 Z 当 Y（y<->z） ---
# df_sw = df.copy()
# df_sw["y"], df_sw["z"] = df["z"], df["y"]

# # --- 4) 重新计算交换后的 box bounds（关键：避免越界 wrap 导致“对不齐”） ---
# xlo, xhi = df_sw["x"].min() - PAD, df_sw["x"].max() + PAD
# ylo, yhi = df_sw["y"].min() - PAD, df_sw["y"].max() + PAD
# zlo, zhi = df_sw["z"].min() - PAD, df_sw["z"].max() + PAD

# # --- 5) 重写文件：保留原 header/sections，只替换 xlo/xhi,ylo/yhi,zlo/zhi，并写新的 Atoms ---
# re_x = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+xlo\s+xhi\s*$")
# re_y = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+ylo\s+yhi\s*$")
# re_z = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+zlo\s+zhi\s*$")

# with OUT_PATH.open("w") as out:
#     # 5.1 写 header 到 Atoms header 行（含 Masses 等都原样保留，但 bounds 行替换）
#     for i in range(atoms_hdr_idx + 1):  # 包含 "Atoms  # atomic" 这一行
#         l = lines[i].rstrip("\n")
#         if re_x.match(l):
#             out.write(f"{xlo:{''}.15f} {xhi:{''}.15f} xlo xhi\n")
#         elif re_y.match(l):
#             out.write(f"{ylo:{''}.15f} {yhi:{''}.15f} ylo yhi\n")
#         elif re_z.match(l):
#             out.write(f"{zlo:{''}.15f} {zhi:{''}.15f} zlo zhi\n")
#         else:
#             out.write(lines[i])

#     # 5.2 确保 Atoms header 后有一个空行（按原文件格式）
#     if atoms_hdr_idx + 1 < len(lines) and lines[atoms_hdr_idx + 1].strip() != "":
#         out.write("\n")

#     # 5.3 写 Atoms 数据（id type x y z）
#     #     注意：id/type 保持整数，坐标用 FLOAT_FMT
#     df_out = df_sw[["id", "type", "x", "y", "z"]].copy()
#     df_out["id"] = df_out["id"].astype(int)
#     df_out["type"] = df_out["type"].astype(int)

#     df_out.to_csv(
#         out,
#         sep=" ",
#         header=False,
#         index=False,
#         float_format=FLOAT_FMT,
#     )

# print("Wrote:", OUT_PATH)
# print("New bounds:")
# print("xlo xhi:", xlo, xhi)
# print("ylo yhi:", ylo, yhi)
# print("zlo zhi:", zlo, zhi)


/Users/lvmeizhong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/lvmeizhong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/var/folders/06/hmnjzdgn7vgcwblryw9g856c0000gn/T/ipykernel_29051/1906020755.py:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


Wrote: /Users/lvmeizhong/Downloads/edge_diamond_glide_set_dissociated_swapYZ.lmp
New bounds:
xlo xhi: -75.0470767729 73.787668008
ylo yhi: -74.7199654502 82.6373232656
zlo zhi: -74.14077634019999 73.6231741509


In [7]:
import re
from pathlib import Path

import pandas as pd

# =========================
# USER CONFIG
# =========================
IN_PATH  = Path("/Users/lvmeizhong/Downloads/edge_diamond_glide_set.lmp")
OUT_PATH = IN_PATH.with_name(IN_PATH.stem + "_swapYZ.lmp")

PAD = 1e-6
FLOAT_FMT = "%.13f"
# =========================

with IN_PATH.open("r") as f:
    lines = f.readlines()

# --- NEW: parse natoms from header (one added block) ---
natoms = None
re_natoms = re.compile(r"^\s*(\d+)\s+atoms\s*$")
for l in lines[:2000]:  # header 一般很短，扫前 2000 行足够
    m = re_natoms.match(l.strip())
    if m:
        natoms = int(m.group(1))
        break
assert natoms is not None, "Header 里没找到 'N atoms'"

# --- 1) 找到 Atoms 段起始行号 ---
atoms_hdr_idx = None
for i, l in enumerate(lines):
    if l.strip().startswith("Atoms"):
        atoms_hdr_idx = i
        break
assert atoms_hdr_idx is not None, "没找到 'Atoms' 段"

atoms_data_start = atoms_hdr_idx + 2  # 假设 Atoms header 后有空行

# --- 2) 读 Atoms：关键是一行 nrows=natoms（就这行最重要） ---
df = pd.read_csv(
    IN_PATH,
    delim_whitespace=True,
    header=None,
    names=["id", "type", "x", "y", "z"],
    skiprows=atoms_data_start,
    nrows=natoms,              # <<< 关键：只读 natoms 行 :contentReference[oaicite:3]{index=3}
    comment="#",
    engine="python",
)

# --- 3) swap y<->z ---
df_sw = df.copy()
df_sw["y"], df_sw["z"] = df["z"], df["y"]

# --- 4) 重新计算 bounds ---
xlo, xhi = df_sw["x"].min() - PAD, df_sw["x"].max() + PAD
ylo, yhi = df_sw["y"].min() - PAD, df_sw["y"].max() + PAD
zlo, zhi = df_sw["z"].min() - PAD, df_sw["z"].max() + PAD

re_x = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+xlo\s+xhi\s*$")
re_y = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+ylo\s+yhi\s*$")
re_z = re.compile(r"^\s*([+-]?\d.*)\s+([+-]?\d.*)\s+zlo\s+zhi\s*$")

with OUT_PATH.open("w") as out:
    # 写 header 到 Atoms header 行（含 Atoms 行本身）
    for i in range(atoms_hdr_idx + 1):
        l = lines[i].rstrip("\n")
        if re_x.match(l):
            out.write(f"{xlo:.15f} {xhi:.15f} xlo xhi\n")
        elif re_y.match(l):
            out.write(f"{ylo:.15f} {yhi:.15f} ylo yhi\n")
        elif re_z.match(l):
            out.write(f"{zlo:.15f} {zhi:.15f} zlo zhi\n")
        else:
            out.write(lines[i])

    # 更稳：无条件补一个空行（LAMMPS 数据文件空行很重要）:contentReference[oaicite:4]{index=4}
    out.write("\n")

    df_out = df_sw[["id", "type", "x", "y", "z"]].copy()
    df_out["id"] = df_out["id"].astype(int)
    df_out["type"] = df_out["type"].astype(int)

    df_out.to_csv(out, sep=" ", header=False, index=False, float_format=FLOAT_FMT)

    # 关键：把原文件 Atoms 段后面的内容原样接回去（如果有的话）
    rest_start = atoms_data_start + natoms
    out.write("\n")
    out.writelines(lines[rest_start:])

print("Wrote:", OUT_PATH)
print("natoms:", natoms)


/var/folders/06/hmnjzdgn7vgcwblryw9g856c0000gn/T/ipykernel_29051/3669369442.py:39: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


Wrote: /Users/lvmeizhong/Downloads/edge_diamond_glide_set_swapYZ.lmp
natoms: 616824


In [1]:
import csv
from pathlib import Path

import numpy as np

# =========================
# USER CONFIG
# =========================
DREAM3D_PATH = Path("/Users/lvmeizhong/Downloads/t0_Meshed_MoreFeature.dream3d")   # 你的 .dream3d/.h5
OUT_CSV      = DREAM3D_PATH.with_suffix("").with_name(DREAM3D_PATH.stem + "_hx70-200_hy70-200_hz150-500.csv")

HX0, HX1 = 70, 200
HY0, HY1 = 70, 200
HZ0, HZ1 = 150, 500

# 如果你的 Frame 类不在 python path，改成你实际 import 路径
# from expoly.frames import Frame
from frames import Frame  # <- 例如你把那份 frames.py 放在当前目录

# =========================

# 1) 读 HDF5：FeatureIds + DIMENSIONS
f = Frame(
    path=DREAM3D_PATH,
    prefer_groups=["CellData", "CellFeatureData", "_SIMPL_GEOMETRY"],  # 和你代码默认一致
    mapping={"GrainId": "FeatureIds", "Dimension": "DIMENSIONS"},       # 只取我们需要的
)

dim = np.asarray(f.Dimension).reshape(-1)
HX, HY, HZ = int(dim[0]), int(dim[1]), int(dim[2])

gid = np.asarray(f.GrainId)
# Dream3D/SIMPL 常见：FeatureIds 是长度 HX*HY*HZ 的 1D 数组
if gid.ndim == 1:
    assert gid.size == HX * HY * HZ, f"FeatureIds size={gid.size} != HX*HY*HZ={HX*HY*HZ}"
    grid = gid.reshape((HZ, HY, HX), order="C")   # grid[hz,hy,hx]
elif gid.ndim == 3:
    # 如果已经是 3D，就尽量对齐到 (HZ,HY,HX)
    if gid.shape == (HZ, HY, HX):
        grid = gid
    elif gid.shape == (HX, HY, HZ):
        grid = np.transpose(gid, (2, 1, 0))       # -> (HZ,HY,HX)
    else:
        raise ValueError(f"Unexpected GrainId shape: {gid.shape}, expected {(HZ,HY,HX)} (or flatten)")
else:
    raise ValueError(f"Unexpected GrainId ndim: {gid.ndim}")

# 2) 边界检查
assert 0 <= HX0 < HX1 <= HX, (HX0, HX1, HX)
assert 0 <= HY0 < HY1 <= HY, (HY0, HY1, HY)
assert 0 <= HZ0 < HZ1 <= HZ, (HZ0, HZ1, HZ)

# 3) 切片并流式写 CSV（避免一次性建 5.9M 行 dataframe）
with OUT_CSV.open("w", newline="") as fp:
    w = csv.writer(fp, delimiter=" ")
    w.writerow(["HZ", "HY", "HX", "grain-ID"])

    # 按 hz 一层一层写（每层 (HY1-HY0)*(HX1-HX0) 行）
    hx_vals = np.arange(HX0, HX1, dtype=np.int32)
    hy_vals = np.arange(HY0, HY1, dtype=np.int32)

    for hz in range(HZ0, HZ1):
        slab = grid[hz, HY0:HY1, HX0:HX1].astype(np.int64, copy=False)  # shape (hy, hx)
        # 逐行写：每行是 (HZ, HY, HX, grain-ID)
        for iy, hy in enumerate(hy_vals):
            row_gid = slab[iy, :]
            # 写这一行的所有 hx
            # (如果你想更快，可改成 fp.write 批量拼接字符串；这个版本先清晰稳妥)
            for ix, hx in enumerate(hx_vals):
                w.writerow([hz, int(hy), int(hx), int(row_gid[ix])])

print("Wrote:", OUT_CSV)
print("Grid dim (HX,HY,HZ) =", (HX, HY, HZ))
print("Kept voxels =", (HX1-HX0)*(HY1-HY0)*(HZ1-HZ0))


/Users/lvmeizhong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/lvmeizhong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


ImportError: attempted relative import with no known parent package